In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from astropy.io import ascii
from astropy.table import Table
import athena_read as ar
from glob import glob
from Constants import Constants
import planet_wind_utils as pw
import seaborn as sns
c = Constants()
import deepdish as dd

%matplotlib inline

# set some global options
plt.rcParams['figure.figsize'] = (6,5)
plt.rcParams['legend.frameon'] = False
plt.rcParams['legend.fontsize'] = 14
plt.rcParams['legend.borderpad'] = 0.2
plt.rcParams['legend.labelspacing'] = 0.2
plt.rcParams['legend.handletextpad'] = 0.2
plt.rcParams['font.family'] = 'stixgeneral'
plt.rcParams['mathtext.fontset'] = 'stix'
plt.rcParams['font.size'] = 16

In [ ]:
import sys
sys.path.append("./planet-wind-rt/")
import mesh_tools as mt

import importlib
importlib.reload(mt)

def makesphere(x, y, z, radius, resolution=10):
    """Return the coordinates for plotting a sphere centered at (x,y,z)"""
    u, v = np.mgrid[0:2*np.pi:resolution*2j, 0:np.pi:resolution*1j]
    X = radius * np.cos(u)*np.sin(v) + x
    Y = radius * np.sin(u)*np.sin(v) + y
    Z = radius * np.cos(v) + z
    return (X, Y, Z)



In [ ]:
import plotly.graph_objects as go 

def make_3d_plot(base_dir,filename_out,nxyz=200,mylevel=1):
    orb = pw.read_trackfile(base_dir+"pm_trackfile.dat")
    print( "ORB: ... ", orb.colnames)

    myfile = base_dir+"PW.out1.00003.athdf"

    # NOTE: read @level=None to get full resolution. 
    d = pw.read_data(myfile,orb,
                     level=mylevel,
                     get_cartesian=True,
                     x1_min = 7e10, x1_max = 0.15*c.au
                    )

    t= d['Time']
    rcom,vcom = pw.rcom_vcom(orb,t)
    x2,y2,z2 = pw.pos_secondary(orb,t)
    
    # define a uniform mesh 
    #nxyz = 200
    lim = 0.05*c.au
    xx,yy,zz,r,th,ph = mt.get_uniform_meshgrid(lim,npoints=nxyz,center=[0,0,0])

    # define interpolating functions & get data
    r0_interp = mt.get_interp_function(d,"r0")
    r0_vals = r0_interp((ph,th,r))


    # get a surface mesh object 
    verts,faces, centroids, areas, normals = mt.get_marching_cubes_mesh(xx,yy,zz,r0_vals,0.005,step_size=1)
    verts1,faces1, centroids1, areas1, normals1 = mt.get_marching_cubes_mesh(xx,yy,zz,r0_vals,0.995,step_size=1)

    # Observer vector
    zf = 2.15
    n0 = np.array([-0.7,-1,0.35])
    n0 = n0/np.linalg.norm(n0)
    camera = dict(
        eye=dict(x=n0[0]*zf, y=n0[1]*zf, z=n0[2]*zf)
    )
    #layout = go.Layout(scene=dict(aspectmode='data',camera=camera))
    myrange=np.array([-0.05,0.05])
    myticks = [-0.025,0,0.025]
    layout=go.Layout(scene = dict(xaxis = dict(range=myrange,showgrid=True,showbackground=False,title='x [au]',tickvals=myticks,gridcolor='grey'),
                                  yaxis = dict(range=myrange,showgrid=True,showbackground=False,title='y [au]',tickvals=myticks,gridcolor='grey'),
                                  zaxis = dict(range=myrange,showgrid=True,showbackground=False,title='z [au]',tickvals=myticks,gridcolor='grey'),
                                  aspectmode='data',
                                  camera=camera),
                    margin=dict(r=0, l=0, b=0, t=0) )

    # planet
    xp,yp,zp=makesphere(x2/c.au,y2/c.au,z2/c.au,2*7e9/c.au,resolution=30)
    xs,ys,zs=makesphere(0,0,0,7e10/c.au,resolution=30)

    mycm = 'plasma_r'


    fig = go.Figure(
        data=[go.Mesh3d(
            x=verts[:,0]/c.au,
            y=verts[:,1]/c.au,
            z=verts[:,2]/c.au,
            i=faces[:,0],
            j=faces[:,1],
            k=faces[:,2],
            intensity=np.ones_like(verts[:,0])*0.2,
            cmin=0,cmax=1,
            #cmin=-2e7,cmax=2e7,opacity=1),
            colorscale=mycm,
            opacity=0.6,
            visible=True,showscale=False
            #lighting=dict(ambient=1),
            #lightposition=dict(x=0,y=0,z=0)
        ), 
        go.Mesh3d(
            x=verts1[:,0]/c.au,
            y=verts1[:,1]/c.au,
            z=verts1[:,2]/c.au,
            i=faces1[:,0],
            j=faces1[:,1],
            k=faces1[:,2],
            intensity=np.ones_like(verts1[:,0])*0.8,
            cmin=0,cmax=1,colorscale=mycm,
            opacity=0.6,
            #lighting=dict(ambient=1),
            #lightposition=dict(x=0,y=0,z=0),
            visible=True,showscale=False
             ),


             go.Surface(x=xp,y=yp,z=zp,
                      showscale=False,
                      surfacecolor=np.zeros_like(zp),
                      colorscale='Reds',
                      lighting=dict(ambient=1)),
              go.Surface(x=xs,y=ys,z=zs,
                      showscale=False,
                      surfacecolor=np.zeros_like(zs),
                      colorscale='Reds',
                      lighting=dict(ambient=1)),
             ],
         layout=layout)

    #fig.show()
    fig.write_image(filename_out)
    

In [ ]:
make_3d_plot('/Volumes/DATAVolume3/athenaruns/streams/grida/a0.025/l4r1/','test3d.svg',nxyz=200,mylevel=1)

In [ ]:
from astropy.convolution import convolve, Box1DKernel

def get_sigmab(d,sep,rp,nanthresh=1e-13):
    b = (d['x3v']-np.pi)*sep
    sigma = np.sum(d['r0']*d['rho']*d['gdr'],axis=2)[:,0]
    sigma=np.where(sigma>nanthresh,sigma,np.nan)
    #sigma=np.where(np.abs(b)>rp,sigma,0)
    velr = np.sum(d['r0']*d['rho']*d['gdr']*-d['vel1'],axis=2)[:,0]/sigma
    sel = np.abs(b)>rp
    return b[sel],sigma[sel],velr[sel]


def read_get_sigmab(base_dir,fn,level,x1_min,x1_max,x3range=np.pi,nanthresh=1.e-13,
                   makeplot=True):
    """ read file from directory and compute column values """
    orb = pw.read_trackfile(base_dir+"pm_trackfile.dat")
    x2sliceval = pw.get_midplane_theta(base_dir+fn,level=level)
    sep = orb['sep'][0]

    d = pw.read_data(base_dir+fn,orb,
                     level=level,
                     get_cartesian=True,
                     x2_min=x2sliceval,x2_max=x2sliceval,
                     x1_min = x1_min, 
                     x1_max = x1_max,
                     x3_min = np.pi - x3range,
                     x3_max = np.pi + x3range,
                     gamma=1.0001)
    d1 = d['x1f'][1:] - d['x1f'][:-1]
    d['gdr']=np.broadcast_to(d1,(len(d['x3v']),len(d['x2v']),len(d['x1v'])) )

 
    
    # parse filename to get rp
    rp = int(base_dir[-2:-1])
    rp=np.where(rp==5,0.5,rp)
    rp*=7e9
    
    b,sigma,velr  = get_sigmab(d,sep,rp,nanthresh=nanthresh)
  
    return d,sep,b,sigma,velr


def sliceplot(d,sep):
    
    plt.figure(figsize=(5,4) )
    plt.pcolormesh(d['x'][:,0,:]/c.au,
                   d['y'][:,0,:]/c.au,
                   np.log10(d['rho'])[:,0,:] ,
                   cmap=plt.cm.Blues,vmax=-16,vmin=-22)
    plt.colorbar(label=r"$\log_{10} \left( \rho \right)$",extend='both')

    plt.axis('equal')

    lim = sep*2
    plt.xlim(-lim/c.au,lim/c.au)
    plt.ylim(-lim/c.au,lim/c.au)
    plt.xlabel('$x$ [au]')
    plt.ylabel('$y$ [au]')
    #plt.show()
    
    
    
def lineplot(sep,b,sigma,vr):
    db = b[1]-b[0]
    width = 2*c.rsun/db
    sigmac = convolve(sigma,Box1DKernel(width),nan_treatment='fill',fill_value=np.nan)
    vrc = convolve(vr,Box1DKernel(width) ,nan_treatment='fill',fill_value=np.nan)


    plim=7e10/sep/(2*np.pi)*5

    plt.figure()
    plt.subplot(211)
    plt.semilogy(-b/sep/(2*np.pi),sigma,'-',lw=2,color='k')
    plt.semilogy(-b/sep/(2*np.pi),sigmac,'--',lw=2,color='r')
    plt.xlim(-plim,plim)
    plt.xticks(visible=False)
    plt.axvline(0,zorder=0,color='grey')
    plt.axvline(-7e10/sep/(2*np.pi),zorder=0,color='grey',ls=':')
    plt.axvline(7e10/sep/(2*np.pi),zorder=0,color='grey',ls=':')
    plt.ylabel(r"$\Sigma$ [g cm$^{-2}$]")

    plt.subplot(212)
    plt.plot(-b/sep/(2*np.pi),vr/1e5,'-',lw=2,color='k')
    plt.plot(-b/sep/(2*np.pi),vrc/1e5,'--',lw=2,color='r')
    plt.plot(-b/sep/(2*np.pi),np.sin(-b/sep) * np.sqrt( c.G*c.msun/sep ) / 1.e5,zorder=0,label='planet frame' )
    plt.xlim(-plim,plim) 
    plt.axhline(0,ls='--',color='grey',zorder=0)
    plt.axvline(0,zorder=0,color='grey')
    plt.axvline(-7e10/sep/(2*np.pi),zorder=0,color='grey',ls=':')
    plt.axvline(7e10/sep/(2*np.pi),zorder=0,color='grey',ls=':')
    plt.ylim(-30,30)
    plt.ylabel(r"$v_{\rm los}$ [km s$^{-1}$]")
    plt.xlabel('phase')
    
    plt.subplots_adjust(hspace=0)
    #plt.show()    

def makefig(d,sep,b,sigma,vr):

    fig = plt.figure(figsize=(5.5,8))
    f1,f2 = fig.subfigures(nrows=2,ncols=1,height_ratios=[1.5,1])
    f1.subplots()
    plt.pcolormesh(d['x'][:,0,:]/c.au,
                   d['y'][:,0,:]/c.au,
                   np.log10(d['rho']*d['r0'])[:,0,:] ,
                   cmap=plt.cm.Blues,vmax=-16,vmin=-22)
    plt.colorbar(label=r"$\log_{10} \left( \rho r_0\right)$",extend='both')

    plt.axis('equal')

    lim = sep*2
    plt.xlim(-lim/c.au,lim/c.au)
    plt.ylim(-lim/c.au,lim/c.au)
    plt.xlabel('$x$ [au]')
    plt.ylabel('$y$ [au]')



    db = b[1]-b[0]
    width = 2*c.rsun/db
    sigmac = convolve(sigma,Box1DKernel(width),nan_treatment='fill',fill_value=np.nan)
    vrc = convolve(vr,Box1DKernel(width) ,nan_treatment='fill',fill_value=np.nan)


    plim=7e10/sep/(2*np.pi)*5
    

    a1,a2 = f2.subplots(2,1)
    a1.semilogy(-b/sep/(2*np.pi),sigma,'-',lw=2,color='k')
    a1.semilogy(-b/sep/(2*np.pi),sigmac,'--',lw=2,color='r')
    a1.set_xlim(-plim,plim)
    #a1.set_xticks(visible=False)
    a1.axvline(0,zorder=0,color='grey')
    a1.axvline(-7e10/sep/(2*np.pi),zorder=0,color='grey',ls=':')
    a1.axvline(7e10/sep/(2*np.pi),zorder=0,color='grey',ls=':')
    a1.set_ylabel(r"$\Sigma$ [g cm$^{-2}$]")


    a2.plot(-b/sep/(2*np.pi),vr/1e5,'-',lw=2,color='k')
    a2.plot(-b/sep/(2*np.pi),vrc/1e5,'--',lw=2,color='r')
    a2.plot(-b/sep/(2*np.pi),np.sin(-b/sep) * np.sqrt( c.G*c.msun/sep ) / 1.e5,zorder=0,label='planet frame' )
    a2.set_xlim(-plim,plim) 
    a2.axhline(0,ls='--',color='grey',zorder=0)
    a2.axvline(0,zorder=0,color='grey')
    a2.axvline(-7e10/sep/(2*np.pi),zorder=0,color='grey',ls=':')
    a2.axvline(7e10/sep/(2*np.pi),zorder=0,color='grey',ls=':')
    a2.set_ylim(-100,100)
    a2.set_ylabel(r"$v_{\rm los}$ [km s$^{-1}$]")
    a2.set_xlabel('phase')

    f2.subplots_adjust(hspace=0,right=0.8)

In [ ]:
d,sep,b,sigma,vr = read_get_sigmab(base_dir="/Volumes/DATAVolume3/athenaruns/streams/grida/a0.025/l4r1/",
                             fn="PW.out1.00003.athdf",
                             level=3,
                             x1_min=7e10,
                             x1_max=0.1*c.au*2,
                             x3range = np.pi, #np.pi/2.5,
                             nanthresh=1e-11)


makefig(d,sep,b,sigma,vr)

In [ ]:
d,sep,b,sigma,vr = read_get_sigmab(base_dir="/Volumes/DATAVolume3/athenaruns/streams/grida/a0.05/l8r2/",
                             fn="PW.out1.00003.athdf",
                             level=3,
                             x1_min=7e10,
                             x1_max=0.1*c.au*2,
                             x3range = np.pi, #np.pi/2.5,
                             nanthresh=1e-11)


makefig(d,sep,b,sigma,vr)